In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
import os

In [6]:
val = []

for i in os.listdir():
  if "ldpc" in i:
    print("Compiling file :", i)
    conv_0 = pd.read_csv(i, header=None)
    for j in range(100):
      val.append(conv_0.iloc[:774, j].values)

Compiling file : ldpc_6snr.csv
Compiling file : ldpc_9snr.csv
Compiling file : ldpc_5snr.csv
Compiling file : ldpc_7snr.csv
Compiling file : ldpc_10snr.csv
Compiling file : ldpc_2snr.csv
Compiling file : ldpc_3snr.csv
Compiling file : ldpc_8snr.csv
Compiling file : ldpc_4snr.csv
Compiling file : ldpc_0snr.csv
Compiling file : ldpc_1snr.csv


In [46]:
X_data_conn = []
x_data = pd.read_csv('final-data-bpsk.csv', header=None)
for i in range(len(x_data)):
  X_data_ext = [[x] for x in x_data.iloc[i, :]]
  X_data_conn.append(X_data_ext)

for i in val:
  X_data_conn.append([[x] for x in i])

X_data_conn = np.asarray(X_data_conn).astype(np.float32)
X_data_conn.shape

(5296, 774, 1)

In [32]:
y_label = pd.read_csv("final-variable-bpsk.csv").values
y_label.shape

(4195, 1)

In [51]:
y_label = pd.read_csv("final-variable-bpsk.csv").values
Y = []
for i in y_label:
  Y.append(i.tolist()[0])

for i in range(len(val)):
  Y.append(4)


In [47]:
X_data = X_data_conn[1:]
X_data.shape

(5295, 774, 1)

In [53]:
y_data = np.asarray(Y)
print(y_data)

[0. 0. 0. ... 4. 4. 4.]


In [61]:
inp_layer = keras.layers.Input(shape=(774,1))

#conv1-64, 3x1
x = keras.layers.Conv1D(filters=128, kernel_size=3)(inp_layer)
x_act = keras.layers.Activation(tf.nn.relu)(x)

#conv1-64, 3x1
x = keras.layers.Conv1D(filters=64, kernel_size=3)(x_act)
x_act = keras.layers.Activation(tf.nn.relu)(x)

#conv1-128, 3x1
x = keras.layers.Conv1D(filters=128, kernel_size=3)(x_act)
x_act = keras.layers.Activation(tf.nn.relu)(x)

#global average pooling
gap = keras.layers.GlobalAveragePooling1D()(x_act)

#dense
d1 = keras.layers.Dense(128, activation="relu")(gap)
d2 = keras.layers.Dense(64, activation="relu")(d1)
out_layer = keras.layers.Dense(5, activation="sigmoid")(d2)

model_test1 = keras.Model(inp_layer, out_layer)

model_test1.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 774, 1)]          0         
                                                                 
 conv1d_9 (Conv1D)           (None, 772, 128)          512       
                                                                 
 activation_9 (Activation)   (None, 772, 128)          0         
                                                                 
 conv1d_10 (Conv1D)          (None, 770, 64)           24640     
                                                                 
 activation_10 (Activation)  (None, 770, 64)           0         
                                                                 
 conv1d_11 (Conv1D)          (None, 768, 128)          24704     
                                                                 
 activation_11 (Activation)  (None, 768, 128)          0   

In [62]:
model_test1.compile(
    optimizer="adam",
    loss = "sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [54]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, shuffle=True, stratify=y_data)

In [55]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((3706, 774, 1), (1589, 774, 1), (3706,), (1589,))

In [56]:
x_t, x_v, y_t, y_v = train_test_split(x_test, y_test, test_size=0.5, shuffle=True, stratify=y_test)

In [57]:
x_t.shape, x_v.shape, y_t.shape, y_v.shape

((794, 774, 1), (795, 774, 1), (794,), (795,))

In [63]:
callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath="best.h5"),
             tf.keras.callbacks.TensorBoard(log_dir="logs")]

In [64]:
x_train.shape

(3706, 774, 1)

In [65]:
model_test1.fit(x_train, y_train, epochs=200, validation_data=(x_v, y_v), callbacks=callbacks)

Epoch 1/200
116/116 [==============================] - 28s 229ms/step - loss: 0.9330 - accuracy: 0.5148 - val_loss: 0.7296 - val_accuracy: 0.5950
Epoch 2/200
116/116 [==============================] - 29s 248ms/step - loss: 0.7182 - accuracy: 0.6039 - val_loss: 0.7297 - val_accuracy: 0.6277
Epoch 3/200
116/116 [==============================] - 25s 215ms/step - loss: 0.7160 - accuracy: 0.6141 - val_loss: 0.7061 - val_accuracy: 0.6277
Epoch 4/200
116/116 [==============================] - 26s 222ms/step - loss: 0.7139 - accuracy: 0.6074 - val_loss: 0.7052 - val_accuracy: 0.5987
Epoch 5/200
116/116 [==============================] - 27s 231ms/step - loss: 0.7103 - accuracy: 0.6104 - val_loss: 0.7034 - val_accuracy: 0.6088
Epoch 6/200
116/116 [==============================] - 27s 232ms/step - loss: 0.7083 - accuracy: 0.6128 - val_loss: 0.6983 - val_accuracy: 0.6264
Epoch 7/200
116/116 [==============================] - 26s 223ms/step - loss: 0.7106 - accuracy: 0.6098 - val_loss: 0.6962 -

In [66]:
model_test1.evaluate(x_t, y_t)

25/25 [==============================] - 1s 55ms/step - loss: 0.2968 - accuracy: 0.8866


[0.29680877923965454, 0.8866498470306396]

In [67]:
model_test1.save("model_ldpc_within_bpsk.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
!zip -r /content/logs.zip /content/logs

  adding: content/logs/ (stored 0%)
  adding: content/logs/validation/ (stored 0%)
  adding: content/logs/validation/events.out.tfevents.1703003851.5a7dddfa4241.624.1.v2 (deflated 79%)
  adding: content/logs/train/ (stored 0%)
  adding: content/logs/train/events.out.tfevents.1703003833.5a7dddfa4241.624.0.v2 (deflated 86%)


In [ ]:
data = np.random.randint(0, 2, 774)
data = np.array([[x] for x in data])
data.shape

(774, 1)

In [ ]:
data = np.expand_dims(data, axis=0)
data.shape

(1, 774, 1)

In [ ]:
np.argmax(model_test1.predict(data)[0])

1/1 [==============================] - 0s 39ms/step


0

In [ ]:
model_test1.save("model")

In [ ]:
!zip -r /content/model.zip /content/model

  adding: content/model/ (stored 0%)
  adding: content/model/fingerprint.pb (stored 0%)
  adding: content/model/keras_metadata.pb (deflated 91%)
  adding: content/model/saved_model.pb (deflated 88%)
  adding: content/model/variables/ (stored 0%)
  adding: content/model/variables/variables.index (deflated 65%)
  adding: content/model/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: content/model/assets/ (stored 0%)


array([[ -6.2288  ],
       [-10.852   ],
       [ -7.1719  ],
       [ -9.5958  ],
       [ -6.8879  ],
       [  8.1482  ],
       [ -7.4452  ],
       [-10.363   ],
       [ -2.3218  ],
       [ 12.854   ],
       [  6.0795  ],
       [  8.5608  ],
       [ -8.6243  ],
       [  7.168   ],
       [  4.4817  ],
       [  5.4672  ],
       [-13.134   ],
       [ -7.6476  ],
       [  9.5916  ],
       [  9.3889  ],
       [ -8.1082  ],
       [ -8.5122  ],
       [ -4.7436  ],
       [ -6.1975  ],
       [ -8.6047  ],
       [-10.513   ],
       [  3.7734  ],
       [ -5.6958  ],
       [-13.235   ],
       [ -1.1011  ],
       [  5.4043  ],
       [  3.237   ],
       [ -2.2656  ],
       [ -8.3731  ],
       [  4.7482  ],
       [ 17.869   ],
       [ -9.0015  ],
       [  4.7875  ],
       [-10.717   ],
       [ -5.0532  ],
       [  9.0479  ],
       [ -8.2392  ],
       [-12.458   ],
       [  1.0485  ],
       [-11.453   ],
       [ -8.8648  ],
       [ -9.3906  ],
       [  5.4